<a href="https://colab.research.google.com/github/Ignas12345/Magistras_antras_dublis/blob/main/knygutes/duomenu_atsisiuntimai/naudojant_GenomicDataCommons/klinikiniu_duomenu_atsisiuntimas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Šita knygute yra skirta įvairaus tipo duomenims atsisiųsti, nes ani bandymai pradėjo nelabai veikti (TCGAbiolinks kažkaip neveikia kai bandau atsiųsti klinikinius ir kitus duomenis...)

In [ ]:
## ----init, results='hide', echo=FALSE, warning=FALSE, message=FALSE-----------
# Install BiocStyle if it's not already installed
if (!requireNamespace("BiocStyle", quietly = TRUE)) {
    if (!requireNamespace("BiocManager", quietly = TRUE))
        install.packages("BiocManager")
    BiocManager::install("BiocStyle")
}

library(knitr)
opts_chunk$set(warning=FALSE, message=FALSE)
library(BiocStyle)  # Load the package
BiocStyle::markdown()

## ----install_bioc, eval=FALSE-------------------------------------------------
if (!require("BiocManager"))
    install.packages("BiocManager")
BiocManager::install('GenomicDataCommons')

## ----libraries, message=FALSE-------------------------------------------------
library(GenomicDataCommons)

## ----statusQS-----------------------------------------------------------------
GenomicDataCommons::status()

## ----statusCheck--------------------------------------------------------------
stopifnot(GenomicDataCommons::status()$status=="OK")

## uztrunka apie 2 minutes

In [2]:
project_id <- 'TCGA-TGCT'

In [4]:
case_ids = cases() |>
    filter( project.project_id == project_id) |>
    results_all() |>
    ids()

In [ ]:
ge_manifest <- files() |>
    filter( cases.project.project_id == project_id) |>
    filter( type == 'gene_expression' ) |>
    filter( analysis.workflow_type == 'STAR - Counts')  |>
    manifest()
head(ge_manifest)

In [5]:
clindat = gdc_clinical(case_ids)
names(clindat)

[1] "demographic" "diagnoses"   "exposures"   "follow_ups"  "main"

In [ ]:
str(clindat$diagnoses)

In [ ]:
str(clindat$diagnoses$submitter_id)
str(clindat$diagnoses$diagnosis_is_primary_disease )
indices <- which(clindat$diagnoses$diagnosis_is_primary_disease == TRUE)


 chr [1:338] "TCGA-2G-AAFE_diagnosis" "TCGA-2G-AAFE_diagnosis3" ...
 logi [1:338] TRUE FALSE FALSE FALSE TRUE TRUE ...


In [ ]:
clindat$diagnoses$submitter_id[indices]
clindat$diagnoses$primary_diagnosis[indices]

In [14]:
indices <- which(clindat$diagnoses$diagnosis_is_primary_disease == TRUE)

df = data.frame(rownames = "Primary diagnosis", #there are a few diagnoses for some of the patients
    t(clindat$diagnoses$primary_diagnosis[indices])
)


colnames(df) = c("feature", clindat$diagnoses$submitter_id[indices])

# Print the result
df


feature,TCGA-2G-AAFE_diagnosis,TCGA-2G-AAGA_diagnosis,TCGA-ZM-AA05_diagnosis,TCGA-2G-AAKD_diagnosis,TCGA-X3-A8G4_diagnosis,TCGA-2G-AAG7_diagnosis,TCGA-2G-AAG5_diagnosis,TCGA-XE-A8H1_diagnosis,TCGA-2G-AAF1_diagnosis,⋯,TCGA-YU-A94L_diagnosis,TCGA-2G-AAG1_diagnosis,TCGA-2G-AAHE_diagnosis,TCGA-XE-AAOI_diagnosis,TCGA-2G-AAI4_diagnosis,TCGA-2G-AAL8_diagnosis,TCGA-2G-AAIB_diagnosis,TCGA-YU-A94H_diagnosis,TCGA-2G-AAFU_diagnosis,TCGA-2G-AAI2_diagnosis
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Primary diagnosis,"Seminoma, NOS","Embryonal carcinoma, NOS","Seminoma, NOS","Embryonal carcinoma, NOS",Mixed germ cell tumor,Mixed germ cell tumor,"Teratoma, benign","Embryonal carcinoma, NOS","Seminoma, NOS",⋯,"Seminoma, NOS",Mixed germ cell tumor,"Seminoma, NOS","Seminoma, NOS","Seminoma, NOS",Mixed germ cell tumor,"Seminoma, NOS","Seminoma, NOS","Seminoma, NOS","Seminoma, NOS"


In [15]:
write.csv(df,"primary_diagnoses")

In [ ]:
not_reported_indices = indices[clindat$diagnoses$primary_diagnosis[indices] == "Not Reported"]
reported_indices = indices[clindat$diagnoses$primary_diagnosis[indices] != "Not Reported"]

In [ ]:
length(reported_indices)

[1] 247

In [ ]:
clindat$diagnoses$case_id

In [ ]:
clindat_df = as_tibble(do.call(rbind, clindat))